In [16]:
import folium
import requests
import datetime
from bs4 import BeautifulSoup


m = folium.Map(
    location=[0, 0],
    zoom_start=2,
)

html_doc = requests.get("http://www.ariss.net/index.cgi?absolute=1").text

soup = BeautifulSoup(html_doc, 'html.parser')
table = soup.findAll("table")[0]
table_rows = table.find_all('tr')
table_rows.pop(0)


final = []
for tr in table_rows:
    data = {}
    td = tr.find_all('td')
    data["call"] = td[0].text.replace(" ", "")
    data["img"] = td[0].find_all("img")[0]["src"]

    try:
        page_messages = requests.get("http://www.findu.com/cgi-bin/msg.cgi?" + data["call"]).text
        soup = BeautifulSoup(page_messages, 'html.parser')
        rows_messages = soup.findAll("table")[0].find_all('tr')
        rows_messages.pop(0)

        data["messages"] = []
        for i in rows_messages:
            t = i.find_all('td')
            #print(t)
            data["messages"].append({"from": t[0].text.replace(" ", ""), "to": t[1].text.replace(" ", ""), "text": t[4].text})

    except Exception:
         data["messages"] = []

    try:
        data["lat"] = float(td[2].text)
        data["lon"] = float(td[3].text)
    except:
        data["lat"] = 0
        data["lon"] = 0
    
    data["UTC heard"] = datetime.datetime.strptime(td[4].text.replace(" ", ""), "%Y%m%d%H%M%S")
    final.append(data)

In [17]:
for i in final:
    pup = '<i>{0} {1}</i>'.format(i["call"], i["UTC heard"])
    icon = folium.features.CustomIcon(i["img"], icon_size=(20, 20))
    folium.Marker([i["lat"], i["lon"]], popup=pup, icon=icon).add_to(m)
m

In [18]:
final[10]["messages"]

[{'from': 'K4KDR-6', 'to': 'WE4B', 'text': ' QSL - 73!!'},
 {'from': 'K4KDR-6',
  'to': 'WE4B',
  'text': ' QSL. Thanks QSO via ISS! 73 de K4KDR Scott FM17es'},
 {'from': 'WE4B', 'to': 'K4KDR-6', 'text': ' hi{Q'},
 {'from': 'N1RCN', 'to': 'WE4B', 'text': ' more people today'},
 {'from': 'N1RCN', 'to': 'WE4B', 'text': ' hi jeff'},
 {'from': 'N1RCN',
  'to': 'WE4B',
  'text': ' [AA] Thanks for the contact ! de n1rcn el87{00'},
 {'from': 'WE4B', 'to': 'N1RCN', 'text': ' hi{P'},
 {'from': 'N1RCN', 'to': 'WE4B', 'text': ' de n1rcn EL87 QSL?'},
 {'from': 'N1RCN', 'to': 'WE4B', 'text': ' good evening'},
 {'from': 'N1RCN', 'to': 'WE4B', 'text': ' de n1rcn via iss'},
 {'from': 'WE4B', 'to': 'K4KDR-6', 'text': ' hi{L'}]